<a href="https://colab.research.google.com/github/sarabert96/Colexification/blob/main/05_gettingCosines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Given a dataframe containing the colexifications, the file returns a new dataframe where the cosine similarity between the pair is calculated.

The file, in the pipeline, is not used.

In [ ]:
pip install fasttext

In [ ]:
import fasttext.util
from google.colab import files

In [ ]:
fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')

In [ ]:
!gdown --id 1YmX2hOvKM5vVRiT7UlCnDKezbYTRZ-kG #df_colexifications

Downloading...
From: https://drive.google.com/uc?id=1YmX2hOvKM5vVRiT7UlCnDKezbYTRZ-kG
To: /content/df_colexifications.csv
7.57MB [00:00, 35.2MB/s]


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('df_colexifications.csv')

In [ ]:
import itertools

def cos_sim(a, b):
	"""Takes 2 vectors a, b and returns the cosine similarity according 
	to the definition of the dot product
	"""
	dot_product = np.dot(a, b)
	norm_a = np.linalg.norm(a)
	norm_b = np.linalg.norm(b)
	return dot_product / (norm_a * norm_b)


# function to write in file
def log(w):
  with open('oov_log.txt', 'a+') as file:
    file.write(w)


# function to handle oov word exception
def trySimilarity(w1, w2, default, model):
  # calculate cosine similarity between two vectors if those exists in the model, else return default --> makes sense? ft deals with OOV
  try:
    v1 = model.get_sentence_vector(w1)
  except:
    log(w1)
    return default
  try:
    v2 = model.get_sentence_vector(w2)
  except:
    log(w2)
    return default
  return cos_sim(v1, v2)


def add_cosine_sim(df,col1,col2,ft_model):
  """
  Enriches dataframe with cosine similarity
  Input: a dataframe, two column names, and a fastText mdodel
  Output: The inputted dataframe with a new column with cosine similarities
  """

  #query the model and if there is an oov word write that word to oov_log.txt and move to the next word

  #add column with cosine sims row-wise

  df['cosine_sim'] = df.apply(lambda row: trySimilarity(row[col1], row[col2], 0, ft_model), axis=1) 

  return df

In [ ]:
df.head(5)

,clics_form,Concepticon_ID.x,Glottocode,Concepticon_Gloss.x,Family,variety,Concepticon_ID.y,Concepticon_Gloss.y
0,s@,1369,hrus1242,gold,Hruso,Hruso Aka Jamiri,1927,bamboo
1,s@,1369,hrus1242,gold,Hruso,Hruso Aka Jamiri,946,blood
2,avir@,1035,miji1239,good,Sino-Tibetan,Dammai Dibin,923,love
3,dzju,1425,hrus1242,green,Hruso,Hruso Aka Jamiri,1424,yellow
4,lah,1277,dakp1242,hand,Sino-Tibetan,Monpa Changprong,639,mountain


In [ ]:
add_cosine_sim(df,'Concepticon_Gloss.x','Concepticon_Gloss.y', ft)

,clics_form,Concepticon_ID.x,Glottocode,Concepticon_Gloss.x,Family,variety,Concepticon_ID.y,Concepticon_Gloss.y,cosine_sim
0,s@,1369,hrus1242,gold,Hruso,Hruso Aka Jamiri,1927,bamboo,0.250825
1,s@,1369,hrus1242,gold,Hruso,Hruso Aka Jamiri,946,blood,0.244084
2,avir@,1035,miji1239,good,Sino-Tibetan,Dammai Dibin,923,love,0.321520
3,dzju,1425,hrus1242,green,Hruso,Hruso Aka Jamiri,1424,yellow,0.682016
4,lah,1277,dakp1242,hand,Sino-Tibetan,Monpa Changprong,639,mountain,0.093907
...,...,...,...,...,...,...,...,...,...
124182,or,688,bong1291,lung,Nuclear Trans New Guinea,bongu,1223,heart,0.449804
124183,eni,763,gira1247,skin,Nuclear Trans New Guinea,girawa,1405,name,0.121962
124184,wus,763,payn1244,skin,Nuclear Trans New Guinea,paynamar,51,sore,0.331377
124185,su,51,kein1239,sore,Nuclear Trans New Guinea,kein,1257,wing,0.084730


In [ ]:
df.head(5)

,clics_form,Concepticon_ID.x,Glottocode,Concepticon_Gloss.x,Family,variety,Concepticon_ID.y,Concepticon_Gloss.y,cosine_sim
0,s@,1369,hrus1242,gold,Hruso,Hruso Aka Jamiri,1927,bamboo,0.250825
1,s@,1369,hrus1242,gold,Hruso,Hruso Aka Jamiri,946,blood,0.244084
2,avir@,1035,miji1239,good,Sino-Tibetan,Dammai Dibin,923,love,0.321520
3,dzju,1425,hrus1242,green,Hruso,Hruso Aka Jamiri,1424,yellow,0.682016
4,lah,1277,dakp1242,hand,Sino-Tibetan,Monpa Changprong,639,mountain,0.093907


In [ ]:
df['Concepticon_pair']=list(zip(df['Concepticon_Gloss.x'],df['Concepticon_Gloss.y']))

In [ ]:
df.head(2)

,clics_form,Concepticon_ID.x,Glottocode,Concepticon_Gloss.x,Family,variety,Concepticon_ID.y,Concepticon_Gloss.y,cosine_sim,Concepticon_pair
0,s@,1369,hrus1242,gold,Hruso,Hruso Aka Jamiri,1927,bamboo,0.250825,"(gold, bamboo)"
1,s@,1369,hrus1242,gold,Hruso,Hruso Aka Jamiri,946,blood,0.244084,"(gold, blood)"


In [ ]:
df.to_csv('df_colex_cosines.csv')

In [ ]:
files.download('df_colex_cosines.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>